Я понял, что не стоит и пытаться обучать модель на всем доступном трейн-датасете, не хватит ни времени, ни вычислительных ресурсов. Остается лишь надеяться, что выборки +- совпадающей по размерам с тестовым датасетом хватит, чтобы получить более или менее приличный результат.

In [43]:
import pandas as pd
pricing_checks_train = pd.read_csv('../input/lenta-pricing/pricing_task/pricing_checks_train.csv', header=None)
pricing_checks_train.columns = [
                                'client_id', 'day', 'shop_id',  'check_id', 'time',  'sku', 'promo_id', 'check_pos', 'num_sales', 'region_name'
]

In [44]:
pricing_checks_train = pricing_checks_train.loc[pricing_checks_train['day']>=20200101]

In [45]:
pricing_checks_train.info()

In [46]:
pricing_checks_test = pd.read_csv('../input/lenta-pricing/pricing_task/pricing_checks_test.csv', header=None)
pricing_checks_test.columns = ['client_id', 'day', 'shop_id', 'check_id', 'time', 'sku',  'promo_id',  'region_name']

In [47]:
pricing_checks_test.info()

In [48]:
pricing_prices_train = pd.read_csv('../input/lenta-pricing/pricing_task/pricing_prices_train.csv', header=None)
pricing_prices_train.columns = ['day', 'shop_id' , 'sku', 'promo_id' , 'reg_price', 'promo_price', 'reg_card_price']
pricing_hierarchy = pd.read_csv('../input/lenta-pricing/pricing_task/pricing_hierarchy.csv', header=None)
pricing_hierarchy.columns = ['sku','class', 'group', 'category', 'subcategory']                                
pricing_shops = pd.read_csv('../input/lenta-pricing/pricing_task/pricing_shops.csv', header=None)
pricing_shops.columns = ['division_id', 'region_id', 'city_id', 'shop_id', 'type_loc_id', 'type_size_id', 'type_format_id', 'type_wealth_id', 'is_store', 'is_active', 'is_ex_billa']

In [49]:
merged_train = pricing_checks_train.merge(pricing_prices_train, how='left')
merged_train = merged_train.merge(pricing_hierarchy, how='left')
merged_train = merged_train.merge(pricing_shops, how='left')

In [50]:
merged_train.loc[:, 'time'] = pd.to_numeric(pd.to_datetime(merged_train.loc[:, 'time']))/1000000000
merged_train.loc[:, 'time'] = merged_train.loc[:, 'time'].astype('float')

Совсем не представляю, что нужно делать с переменной check_pos

In [51]:
merged_train = merged_train.drop('check_pos', axis=1)

In [52]:
merged_train.columns

In [53]:
cols = merged_train.columns

In [54]:
cat_features = [x for x in cols if merged_train[x].dtype=='object']
num_features = [x for x in cols if x not in cat_features]

In [55]:
import numpy as np
print(merged_train[cat_features].columns)
print(merged_train[num_features].columns)
C_train = merged_train[cat_features].values
N_train = merged_train[num_features].drop('num_sales', axis=1).values
y_train = merged_train['num_sales'].values

In [60]:
cols_in_right_order = ['client_id', 'shop_id', 'check_id', 'sku', 'promo_id', 'region_name',
       'class', 'group', 'category', 'subcategory', 'division_id', 'region_id',
       'city_id']+ ['day', 'time', 'num_sales', 'reg_price', 'promo_price',
       'reg_card_price', 'type_loc_id', 'type_size_id', 'type_format_id',
       'type_wealth_id', 'is_store', 'is_active', 'is_ex_billa']

cols_in_right_order[16:19]

Понимаю, не лучшая практика, но доделываю в спешке, поэтому за цену будет отвечать 15: 18 индексы, учту на будущее

In [ ]:
C_train.shape, N_train.shape, y_train.shape

In [ ]:
total_len = C_train.shape[0]
train_border, val_border = int(total_len * 0.75), int(total_len * 0.875)

In [ ]:
C_val, N_val, y_val = C_train[train_border:val_border], N_train[train_border:val_border], y_train[train_border:val_border]
C_test, N_test, y_test = C_train[val_border:], N_train[val_border:], y_train[val_border:]
C_train, N_train, y_train = C_train[:train_border], N_train[:train_border], y_train[:train_border]

In [ ]:
import os
os.mkdir('data')
os.mkdir('data/lenta_training')
np.save('data/lenta_training/C_train.npy', C_train, allow_pickle=True)
np.save('data/lenta_training/N_train.npy', N_train)
np.save('data/lenta_training/y_train.npy', y_train)
np.save('data/lenta_training/C_val.npy', C_val, allow_pickle=True)
np.save('data/lenta_training/N_val.npy', N_val)
np.save('data/lenta_training/y_val.npy', y_val)
np.save('data/lenta_training/C_test.npy', C_test, allow_pickle=True)
np.save('data/lenta_training/N_test.npy', N_test)
np.save('data/lenta_training/y_test.npy', y_test)

In [ ]:
del merged_train, pricing_prices_train, pricing_checks_train

In [ ]:
pricing_prices_test = pd.read_csv('../input/lenta-pricing/pricing_task/pricing_prices_test.csv', header=None)
pricing_prices_test.columns = ['day', 'shop_id' , 'sku', 'promo_id' , 'reg_price', 'promo_price', 'reg_card_price']

In [ ]:
merged_test = pricing_checks_test.merge(pricing_prices_test, how='left')
merged_test = merged_test.merge(pricing_hierarchy, how='left')
merged_test = merged_test.merge(pricing_shops, how='left')

In [ ]:
merged_test.loc[:, 'time'] = pd.to_numeric(pd.to_datetime(merged_test.loc[:, 'time']))/1000000000
merged_test.loc[:, 'time'] = merged_test.loc[:, 'time'].astype('float')

In [ ]:
cols = merged_test.columns
cat_features = [x for x in cols if merged_test[x].dtype=='object']
num_features = [x for x in cols if x not in cat_features]

In [ ]:
C_evaluation = merged_test[cat_features].values
N_evaluation = merged_test[num_features].values

In [36]:
C_evaluation.shape

In [ ]:
os.mkdir('data/lenta_evaluating')
np.save('data/lenta_evaluating/C_eval.npy', C_evaluation, allow_pickle=True)
np.save('data/lenta_evaluating/N_eval.npy', N_evaluation)

In [ ]:
!zip -r output.zip data

In [ ]:
from IPython.display import FileLink
FileLink('output.zip')

Вот что получилось

In [34]:
submission = merged_test[['sku', 'shop_id', 'day']]
submission

In [38]:
preds = np.load('../input/predsv2/preds2.npy')

In [39]:
submission['price_pred'] = preds

In [41]:
submission.to_csv('submission1.csv', index=False)

In [57]:
pd.read_csv('submission1.csv')